# Patrón Bulkhead: Aislamiento de Fallos en Sistemas Distribuidos

## ¿Qué es el patrón Bulkhead?

El patrón **Bulkhead** (literalmente "mamparo" en inglés) es un patrón de diseño de tipo estructural/arquitectónico que busca la resiliencia al **aislar elementos de una aplicación en grupos independientes** para que un fallo en un grupo no afecte a los demás. 

Inspirado en los mamparos estancos de los barcos que evitan que una vía de agua hunda toda la embarcación.

## Ventajas vs. Desventajas

| Ventajas | Desventajas |
|----------|-------------|
| Aislamiento de fallos | Mayor complejidad |
| Mejor disponibilidad | Overhead de recursos |
| Límites claros de recursos | Dificultad para dimensionar grupos |
| Degradación controlada | Posible subutilización |

## Ejemplo: Sistema de Pedidos con Bulkhead

Vamos a implementar un sistema de procesamiento de pedidos donde:
- Procesamiento de pagos y envíos son servicios críticos
- Queremos evitar que fallos en envíos afecten pagos

In [1]:
import threading
import random
import time
from concurrent.futures import ThreadPoolExecutor

## Implementación del Bulkhead

Usaremos **ThreadPoolExecutor separados** para cada servicio, limitando los hilos disponibles por grupo.

In [2]:
class OrderSystem:
    def __init__(self):
        # Bulkhead para pagos - máximo 3 hilos
        self.payment_executor = ThreadPoolExecutor(max_workers=3, thread_name_prefix='payment_')
        
        # Bulkhead para envíos - máximo 2 hilos
        self.shipping_executor = ThreadPoolExecutor(max_workers=2, thread_name_prefix='shipping_')
        
        # Bulkhead para notificaciones - máximo 1 hilo
        self.notification_executor = ThreadPoolExecutor(max_workers=1, thread_name_prefix='notif_')

    def process_payment(self, order_id):
        try:
            print(f"[Pago] Procesando orden {order_id} en hilo {threading.current_thread().name}")
            time.sleep(random.uniform(0.5, 1.5))  # Simula procesamiento
            
            # Simular fallo aleatorio (10% de probabilidad)
            if random.random() < 0.1:
                raise Exception("Error en procesamiento de pago")
                
            print(f"[Pago] Orden {order_id} completada")
            return True
        except Exception as e:
            print(f"[Pago] Error en orden {order_id}: {str(e)}")
            return False

    def process_shipping(self, order_id):
        try:
            print(f"[Envío] Procesando orden {order_id} en hilo {threading.current_thread().name}")
            time.sleep(random.uniform(1, 3))  # Envío tarda más
            
            # Simular fallo aleatorio (20% de probabilidad)
            if random.random() < 0.2:
                raise Exception("Error en sistema de envíos")
                
            print(f"[Envío] Orden {order_id} despachada")
            return True
        except Exception as e:
            print(f"[Envío] Error en orden {order_id}: {str(e)}")
            return False

    def send_notification(self, order_id):
        try:
            print(f"[Notif] Enviando confirmación para orden {order_id}")
            time.sleep(0.5)
            print(f"[Notif] Orden {order_id}: Notificación enviada")
            return True
        except Exception as e:
            print(f"[Notif] Error en orden {order_id}: {str(e)}")
            return False

    def process_order(self, order_id):
        # Procesar pago (en su propio bulkhead)
        payment_future = self.payment_executor.submit(self.process_payment, order_id)
        
        # Procesar envío (en bulkhead separado)
        shipping_future = self.shipping_executor.submit(self.process_shipping, order_id)
        
        # Esperar resultados
        payment_ok = payment_future.result()
        shipping_ok = shipping_future.result()
        
        # Notificación (en bulkhead de baja prioridad)
        if payment_ok and shipping_ok:
            self.notification_executor.submit(self.send_notification, order_id)
        
        return payment_ok and shipping_ok

## Simulación del Sistema
Vamos a probar el sistema con 10 pedidos concurrentes. Observa cómo:
- Los fallos en envíos no afectan pagos
- La saturación de envíos no bloquea notificaciones

In [5]:
def simulate_orders():
    # 1. Inicialización del sistema con Bulkheads
    system = OrderSystem()  # Crea instancia con los ThreadPools aislados
    order_ids = range(1, 11)  # Genera 10 IDs de pedido (1 al 10)
    
    # 2. Ejecución concurrente de pedidos
    with ThreadPoolExecutor(max_workers=10) as executor:  # Pool para simular usuarios concurrentes
        # 3. Envío asíncrono de todas las órdenes
        futures = [executor.submit(system.process_order, oid) for oid in order_ids]
        
        # 4. Recolección de resultados
        for future in futures:
            try:
                future.result(timeout=5)  # Espera máximo 5 segundos por cada orden
            except Exception as e:
                print(f"Error procesando orden: {str(e)}")

In [6]:
if __name__ == "__main__":
    print("=== Inicio simulación Bulkhead ===")
    simulate_orders()
    print("=== Simulación completada ===")


=== Inicio simulación Bulkhead ===
[Pago] Procesando orden 1 en hilo payment__0
[Envío] Procesando orden 1 en hilo shipping__0
[Pago] Procesando orden 2 en hilo payment__1
[Envío] Procesando orden 2 en hilo shipping__1
[Pago] Procesando orden 3 en hilo payment__2
[Pago] Orden 3 completada
[Pago] Procesando orden 4 en hilo payment__2
[Envío] Error en orden 1: Error en sistema de envíos
[Envío] Procesando orden 3 en hilo shipping__0
[Pago] Orden 1 completada
[Pago] Procesando orden 5 en hilo payment__0
[Pago] Orden 2 completada
[Pago] Procesando orden 6 en hilo payment__1
[Pago] Orden 4 completada
[Pago] Procesando orden 7 en hilo payment__2
[Pago] Orden 5 completada
[Pago] Procesando orden 8 en hilo payment__0
[Envío] Error en orden 2: Error en sistema de envíos
[Envío] Procesando orden 4 en hilo shipping__1
[Pago] Orden 6 completada
[Pago] Procesando orden 9 en hilo payment__1
[Pago] Orden 9 completada
[Pago] Procesando orden 10 en hilo payment__1
[Pago] Orden 7 completada
[Pago] Orden

## Análisis de Resultados

Observa en la salida:
1. **Nombres de hilos diferentes** para cada grupo (payment_, shipping_, notif_)
2. **Fallo en un servicio no colapsa otros**
3. **Notificaciones siguen funcionando** aunque envíos fallen
4. **Límite de hilos** respetado por cada grupo

## Escenarios para Usar Bulkhead

1. **Microservicios con distinto SLA**
2. **Sistemas con componentes de distinta criticidad**
3. **Aplicaciones con recursos compartidos limitados**
4. **Prevención de fallos en cascada**